In [ ]:
using CUDA
using DICOM
using KernelAbstractions
using CUDAKernels
using Juliana
using JSON
using PyPlot
using Statistics

# Config

In [ ]:
output_dir = "wed_test"

mkpath(output_dir)

# Helper functions

In [ ]:
convert_to_sp = Juliana.hu_to_sp_factory("/data/user/bellotti_r/semester_project_planning_metrics/src/pyftpp/bin/huToSp.json")

In [ ]:
hu = -1000:1:3095
sp = convert_to_sp.(hu)

fig, ax = subplots()
ax.plot(hu, sp);
ax.set_xlim([-50, 50])
ax.set_ylim([0.8, 1.2])
ax.axhline(1, color=:black)
ax.axvline(0, color=:black)
ax.set_ylabel("relative stopping power")
ax.set_xlabel("HU")

# Renate H&N phantom

In [ ]:
fiona_standalone_bin_path = "/data/user/bellotti_r/semester_project_planning_metrics/src/pyftpp/bin"
fiona_jar_path = "$fiona_standalone_bin_path/ch.psi.ftpp.standalone.planner-1.0.10.jar";

In [ ]:
path_to_phantom = "/data/user/bellotti_r/data_special_cases/renate";
target_dose = 1.f0;

In [ ]:
ct, structures = Juliana.load_dicom_directory(
    path_to_phantom;
    structure_names=["PTV_total_2mm", "BrainStem_Surface"],
);
target = structures["PTV_total_2mm"];

constraints = Juliana.parse_oar_constraints_file(
    "$(path_to_phantom)/constraints.csv",
    target_dose,
    structures,
)
prescriptions = Juliana.Prescriptions([("PTV_total_2mm", target_dose)], constraints);

In [ ]:
@assert ct.grid.origin == [0, 0, 0];

In [ ]:
using StaticArrays

In [ ]:
const HU_AIR = -600;

In [ ]:
gantry_angle = 0f0
couch_angle = 0f0;

In [ ]:
structure_mask_expanded = target.distanceFromStructure .<= 5

@time in_beam_mask = Juliana.calculate_in_beam_mask(
    structure_mask_expanded,
    gantry_angle,
    couch_angle,
    ct.grid,
);
relevant_mask = ((ct.data .> HU_AIR) .|| structure_mask_expanded) .&& (in_beam_mask);

In [ ]:
wed_grid = Juliana.FionaStandalone.get_fiona_dose_calc_grid(ct.grid)
wed_mask = Array{Bool, 3}(undef, wed_grid.size...)
fill!(wed_mask, true)
points, indices = Juliana.mask_to_points_and_indices(wed_grid, wed_mask);

to_keep = []
for j in 1:size(points, 2)
    ct_ind = Juliana.xyz_to_index(points[:, j], ct.grid)
    if relevant_mask[ct_ind...]
        push!(to_keep, j)
    end
end

points = points[:, to_keep]
indices = indices[:, to_keep];

In [ ]:
size(points)

In [ ]:
# gantry_angle = 90.f0;
# couch_angle = 76.f0;

## WED using Juliana

In [ ]:
densities = Juliana.ScalarGrid(
    convert_to_sp.(ct.data),
    ct.grid,
);

In [ ]:
wed = Juliana.calculate_wed(
    densities,
    gantry_angle,
    couch_angle,
    wed_grid,
);

In [ ]:
wed_grid_interpolated = Juliana.interpolate_linearly(wed, ct.grid);

In [ ]:
maximum(wed.data)

In [ ]:
maximum(wed_grid_interpolated.data)

## WED using FIonA

In [ ]:
working_dir = output_dir
mkpath(working_dir);

In [ ]:
Juliana.write_ct_dat_file("$(output_dir)/renate_phantom.dat", ct);

In [ ]:
iso_center = mean(structures["PTV_total_2mm"].points, dims=1)[1:3]
z = Juliana.xyz_to_index(iso_center, ct.grid)[3]

In [ ]:
Dij, optim_points = Juliana.FionaStandalone.calculate_Dij(
    working_dir,
    "$(output_dir)/renate_phantom.dat",
    1,
    target,
    fiona_standalone_bin_path,
    fiona_jar_path,
    wed_grid,
    [gantry_angle],
    [couch_angle],
    [15.0f0],
    log_wed=true,
    optimization_points=points,
);

In [ ]:
wed_fiona_flattened = JSON.parsefile("$(output_dir)/WED_0.json")
wed_fiona = Juliana.ScalarGrid(
    Juliana.flat_vector_to_cube(ct.grid, indices, wed_fiona_flattened),
    ct.grid,
);

In [ ]:
fig, axes = Juliana.plot_distributions(
    ct,
    [
        ["my code WED", relevant_mask],
    ],
    [("PTV", (target, "blue"))],
    1, 512,
    1, 512,
    z,
    1,
    Juliana.build_colorscheme(),
)

In [ ]:
fig, axes = Juliana.plot_distributions(
    ct,
    [
        ["my code WED", wed_grid_interpolated.data],
        ["FIonA WED", wed_fiona.data],
    ],
    [("PTV", (target, "blue"))],
    1, 512,
    1, 512,
    z,
    15,
    Juliana.build_colorscheme(),
)

# Water slab 

In [ ]:
ct, whole_body = Juliana.build_water_slab();

In [ ]:
Juliana.write_ct_dat_file("$(output_dir)/water_phantom.dat", ct)

In [ ]:
ct_new = Juliana.load_ct_dat_file("$(output_dir)/water_phantom.dat")
@assert ct.grid.origin == ct_new.grid.origin
@assert ct.grid.spacing == ct_new.grid.spacing
@assert ct.grid.size == ct_new.grid.size
@assert ct.data == ct_new.data

In [ ]:
densities = Juliana.ScalarGrid(
    convert_to_sp.(ct.data),
    ct.grid,
);

## Calculate WED using Juliana

In [ ]:
ct_mask = similar(whole_body.mask)
fill!(ct_mask, true);

In [ ]:
d_direction = cu(Juliana.angles_to_direction(gantry_angle, couch_angle))

points, indices = Juliana.mask_to_points_and_indices(ct.grid, ct_mask)

wed = Juliana.calculate_wed(
    densities,
    gantry_angle,
    couch_angle,
    ct.grid,
);

## Calculate WED using Fiona

In [ ]:
working_dir = output_dir
mkpath(working_dir)

# optim_grid = Juliana.Grid(
#     ct.grid.spacing,
#     [1.9f0, 1.9f0, 0.4f0],
#     [61, 61, 11],
# )

Dij, optim_points = Juliana.FionaStandalone.calculate_Dij(
    working_dir,
    "$(output_dir)/water_phantom.dat",
    1,
    whole_body,
    fiona_standalone_bin_path,
    fiona_jar_path,
    ct.grid,
    [gantry_angle],
    [couch_angle],
    [15.0f0],
    log_wed=true,
    optimization_points=points,
);

In [ ]:
wed_fiona_flattened = JSON.parsefile("$(output_dir)/WED_0.json")
wed_fiona = Juliana.ScalarGrid(
    Juliana.flat_vector_to_cube(ct.grid, indices, wed_fiona_flattened),
    ct.grid,
);

## Compare

In [ ]:
fig, axes = Juliana.plot_distributions(
    ct,
    [
        ["my code WED", wed.data],
        ["fiona WED", wed_fiona.data],
    ],
    Dict("whole body" => [whole_body, "blue"]),
    6,
    100,
    1,
    100,
    4,
    1.5,
    Juliana.build_colorscheme(),
)

In [ ]:
maximum(wed.data)

In [ ]:
maximum(wed_fiona.data)

In [ ]:
errors = abs.(wed.data .- wed_fiona.data);
maximum(errors)

In [ ]:
fig, axes = plt.subplots(
    1, 3,
    figsize=(8 * 1.3, 4.5),
    gridspec_kw=Dict("width_ratios" => [1, 1, 0.04]),
)
img = axes[1].imshow(wed.data[:, :, 6]')
img = axes[2].imshow(wed.data[:, :, 6]')
fig.colorbar(img, cax=axes[3])

In [ ]:
fig, ax = plt.subplots()

arr = collect(wed.data[40:80, 20:80, 6])
μ = vec(mean(arr, dims=2))
σ = vec(std(arr, dims=2))
ax.errorbar(1:length(μ), μ, yerr=σ, label="Juliana")

arr = collect(wed_fiona.data[40:80, 20:80, 6])
μ = vec(mean(arr, dims=2))
σ = vec(std(arr, dims=2))
ax.errorbar(1:length(μ), μ, yerr=σ, label="Fiona")
ax.legend(loc="lower right")
#ax.set_yscale(:log)

In [ ]:
arr = collect(wed.data[40:80, 20:80, 6])
mean(arr, dims=1), std(arr, dims=1)

In [ ]:
fig, ax = plt.subplots()
img = ax.imshow((wed.data .- wed_fiona.data)[:, :, 6]' .* 1e5, cmap="coolwarm", vmin=-2.0, vmax=2.0)
fig.colorbar(img)

# Alternating bone and water

In [ ]:
ct, whole_body = Juliana.build_alternating_bone_water()

Juliana.write_ct_dat_file("$(output_dir)/water_phantom2.dat", ct)

densities = Juliana.ScalarGrid(
    convert_to_sp.(ct.data),
    ct.grid,
);

## WED using Juliana

In [ ]:
ct_mask = similar(whole_body.mask)
fill!(ct_mask, true);

In [ ]:
points, indices = Juliana.mask_to_points_and_indices(ct.grid, ct_mask);

In [ ]:
wed = Juliana.calculate_wed(
    densities,
    gantry_angle,
    couch_angle,
    ct.grid,
);

## WED using Fiona

In [ ]:
working_dir = output_dir
mkpath(working_dir)

Dij, optim_points = Juliana.FionaStandalone.calculate_Dij(
    working_dir,
    "$(output_dir)/water_phantom2.dat",
    1,
    whole_body,
    fiona_standalone_bin_path,
    fiona_jar_path,
    ct.grid,
    [gantry_angle],
    [couch_angle],
    [15.0f0],
    log_wed=true,
    optimization_points=points,
);

wed_fiona_flattened = JSON.parsefile("$(output_dir)/WED_0.json")
wed_fiona = Juliana.ScalarGrid(
    Juliana.flat_vector_to_cube(ct.grid, indices, wed_fiona_flattened),
    ct.grid,
);

## Compare

In [ ]:
fig, axes = Juliana.plot_distributions(
    ct,
    [
        ["my code WED", wed.data],
        ["fiona WED", wed_fiona.data],
    ],
    Dict("whole body" => [whole_body, "blue"]),
    1,
    100,
    1,
    100,
    7,
    1.2,
    Juliana.build_colorscheme(),
)

In [ ]:
maximum(abs.(wed.data .- wed_fiona.data))

In [ ]:
fig, ax = plt.subplots()
img = ax.imshow((wed.data .- wed_fiona.data)[:, :, 6]' .* 1e4, cmap="coolwarm", vmin=-2., vmax=2.)
fig.colorbar(img)

# Bone gradient

In [ ]:
ct, whole_body = Juliana.build_bone_gradient()

Juliana.write_ct_dat_file("$(output_dir)/water_phantom3.dat", ct)

densities = Juliana.ScalarGrid(
    convert_to_sp.(ct.data),
    ct.grid,
)

## WED using Juliana

In [ ]:
points, indices = Juliana.grid_to_points_and_indices(ct.grid);

In [ ]:
wed = Juliana.calculate_wed(
    densities,
    gantry_angle,
    couch_angle,
    ct.grid,
);

## WED using Fiona

In [ ]:
working_dir = output_dir
mkpath(working_dir)

fiona_standalone_bin_path = "/data/user/bellotti_r/semester_project_planning_metrics/src/pyftpp/bin"
fiona_jar_path = "$fiona_standalone_bin_path/ch.psi.ftpp.standalone.planner-1.0.7.jar";

optim_grid = Juliana.Grid(
    ct.grid.spacing,
    [2f0, 2f0, 0.5f0],
    [61, 61, 11],
)

Dij, optim_points = Juliana.FionaStandalone.calculate_Dij(
    working_dir,
    "$(output_dir)/water_phantom3.dat",
    1,
    whole_body,
    fiona_standalone_bin_path,
    fiona_jar_path,
    optim_grid,
    [0f0],
    [0f0],
    [15.0f0],
    log_wed=true,
    optimization_points=points',
);

wed_fiona_flattened = JSON.parsefile("$(output_dir)/WED_0.json");
wed_fiona = Juliana.flat_vector_to_cube(ct.grid, indices, wed_fiona_flattened);

## Compare

In [ ]:
fig, axes = Juliana.plot_distributions(
    ct,
    [
        ["my code WED", wed],
        ["fiona WED", wed_fiona],
    ],
    Dict("whole body" => [whole_body, "blue"]),
    1,
    100,
    1,
    100,
    7,
    6.5,
    Juliana.build_colorscheme(),
)

In [ ]:
maximum(abs.(wed .- wed_fiona))

In [ ]:
argmax(abs.(wed .- wed_fiona))

In [ ]:
fig, ax = plt.subplots()
img = ax.imshow((wed .- wed_fiona)[:, :, 6]' .* 1e4, cmap="coolwarm", vmin=-2., vmax=2.)
fig.colorbar(img)

## Log the CT

In [ ]:
patient_ID = "phantom_3"
new_patient_ID = "bellotti_r_wed_debug"
patient_name = "$(new_patient_ID)^$(new_patient_ID)"

study_instance_UID = Juliana.get_study_instance_uid(new_patient_ID)
frame_of_reference_UID = "$(study_instance_UID).0"
ct_series_instance_UID = "$(study_instance_UID).1"
structureset_series_instance_UID = "$(study_instance_UID).2"

In [ ]:
ct_datasets = Juliana.ct_to_dicom(
    ct,
    study_instance_UID,
    frame_of_reference_UID,
    ct_series_instance_UID,
    new_patient_ID,
    patient_name,
)
for ds in ct_datasets
    ds.SeriesDescription = "CT Images"
    dcm_write("$(output_dir)/CT.$(ds.InstanceNumber).dcm", ds)
end

# Bone gradient (long)

In [ ]:
ct, whole_body = Juliana.build_bone_gradient_long()

Juliana.write_ct_dat_file("$(output_dir)/water_phantom4.dat", ct)

densities = convert_to_sp.(ct.data)
d_densities = cu(densities);

## WED using Juliana

In [ ]:
d_direction = cu(Juliana.angles_to_direction(0, 0))

points, indices = Juliana.mask_to_points_and_indices(ct.grid, whole_body.mask)
d_points = cu(points)
N = size(d_points, 2)

d_grid = cu(ct.grid)
d_wed_flattened = cu(zeros(Float64, N, 1));

event = Juliana.calculate_wed_simple(
    d_wed_flattened,
    d_densities,
    d_grid,
    d_points,
    d_direction,
    ndrange=(N, 1),
#     STEP_SIZE=0.005,
)
wait(event);

In [ ]:
wed_flattened = collect(d_wed_flattened);
wed = Juliana.flat_vector_to_cube(ct.grid, indices, wed_flattened[:, 1]);

## WED using Fiona

In [ ]:
working_dir = output_dir
mkpath(working_dir)

fiona_standalone_bin_path = "/data/user/bellotti_r/semester_project_planning_metrics/src/pyftpp/bin"
fiona_jar_path = "$fiona_standalone_bin_path/ch.psi.ftpp.standalone.planner-1.0.7.jar";

optim_grid = Juliana.Grid(
    ct.grid.spacing,
    [8f0, 4f0, 1f0],
    [41, 61, 11],
)

Dij, optim_points = Juliana.FionaStandalone.calculate_Dij(
    working_dir,
    "$(output_dir)/water_phantom4.dat",
    1,
    whole_body,
    fiona_standalone_bin_path,
    fiona_jar_path,
    optim_grid,
    [0f0],
    [0f0],
    [15.0f0],
    log_wed=true,
    optimization_points=points',
);

wed_fiona_flattened = JSON.parsefile("$(output_dir)/WED_0.json");
wed_fiona = Juliana.flat_vector_to_cube(ct.grid, indices, wed_fiona_flattened);

## Compare

In [ ]:
fig, axes = Juliana.plot_distributions(
    ct,
    [
        ["my code WED", wed],
        ["fiona WED", wed_fiona],
    ],
    Dict("whole body" => [whole_body, "blue"]),
    1,
    100,
    1,
    100,
    7,
    6.5,
    Juliana.build_colorscheme(),
)

In [ ]:

iy = 20
λ = 1. / 60
-(iy-20.) / λ

In [ ]:
maximum(abs.(wed .- wed_fiona))

In [ ]:
argmax(abs.(wed .- wed_fiona))

In [ ]:
fig, ax = plt.subplots()
img = ax.imshow((wed .- wed_fiona)[:, :, 6]' .* 1e4, cmap="coolwarm", vmin=-2., vmax=2.)
fig.colorbar(img)

# Water slab with cavity

In [ ]:
ct, whole_body = Juliana.build_water_slab_with_cavity()

Juliana.write_ct_dat_file("$(output_dir)/water_phantom5.dat", ct)

densities = convert_to_sp.(ct.data)
d_densities = cu(densities);

## WED using Juliana

In [ ]:
d_direction = cu(Juliana.angles_to_direction(0, 0))

points, indices = Juliana.mask_to_points_and_indices(ct.grid, whole_body.mask)
d_points = cu(points)
N = size(d_points, 2)

d_grid = cu(ct.grid)
d_wed_flattened = cu(zeros(Float64, N, 1));

event = Juliana.calculate_wed_simple(
    d_wed_flattened,
    d_densities,
    d_grid,
    d_points,
    d_direction,
    ndrange=(N, 1),
#     STEP_SIZE=0.005,
)
wait(event);

In [ ]:
wed_flattened = collect(d_wed_flattened);
wed = Juliana.flat_vector_to_cube(ct.grid, indices, wed_flattened[:, 1]);

## WED using Fiona

In [ ]:
working_dir = output_dir
mkpath(working_dir)

fiona_standalone_bin_path = "/data/user/bellotti_r/semester_project_planning_metrics/src/pyftpp/bin"
fiona_jar_path = "$fiona_standalone_bin_path/ch.psi.ftpp.standalone.planner-1.0.7.jar";

optim_grid = Juliana.Grid(
    ct.grid.spacing,
    [2f0, 2f0, 0.5f0],
    [61, 61, 11],
)

Dij, optim_points = Juliana.FionaStandalone.calculate_Dij(
    working_dir,
    "$(output_dir)/water_phantom5.dat",
    1,
    whole_body,
    fiona_standalone_bin_path,
    fiona_jar_path,
    optim_grid,
    [0f0],
    [0f0],
    [15.0f0],
    log_wed=true,
    optimization_points=points',
);

wed_fiona_flattened = JSON.parsefile("$(output_dir)/WED_0.json");
wed_fiona = Juliana.flat_vector_to_cube(ct.grid, indices, wed_fiona_flattened);

## Compare

In [ ]:
fig, axes = Juliana.plot_distributions(
    ct,
    [
        ["my code WED", wed],
        ["fiona WED", wed_fiona],
    ],
    Dict("whole body" => [whole_body, "blue"]),
    1,
    100,
    1,
    100,
    7,
    6.5,
    Juliana.build_colorscheme(),
)

In [ ]:
maximum(abs.(wed .- wed_fiona))

In [ ]:
argmax(abs.(wed .- wed_fiona))

In [ ]:
fig, ax = plt.subplots()
img = ax.imshow((wed .- wed_fiona)[:, :, 6]' .* 1e4, cmap="coolwarm", vmin=-2., vmax=2.)
fig.colorbar(img)